In [68]:
pip install diptest


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [41]:
import rioxarray as rxr
import numpy as np
import xarray as xr
import os
from scipy import stats
import pandas as pd
import pylab as py
import statsmodels.api as sm
import diptest
import geopandas as gpd

In [42]:
basin_path =r'/Volumes/PhD/imagery/masters/output/dNBR/'
out = r'/Volumes/PhD/imagery/masters/output/normality/'

In [43]:
B_name = []
statistic = []
p_value = []

SW_normal = []

DP_statistic = []
DP_p_value = []

DP_normal = []

dip_test_stat = []
dip_test_p_value = []

bimodal = []

Q1 = []
Q3 = []

# sort files in the directory
#basin_path = sorted(basin_path)
for file in os.listdir(basin_path):
    if file.__contains__('.tif'):
        # open the raster file
        basin = rxr.open_rasterio(os.path.join(basin_path, file), masked=True)
        # flatten the array to 1D, remove NaN values
        basin = basin.values.flatten()
        basin = basin[~np.isnan(basin)]
        name = file.split('_')[1]
        #if len(basin) > 5000:
            # create quantile-quantile plot with 95% confidence interval
            #sm.qqplot(basin, line='s', markersize=1)
            #py.title(f'Basin {name} Q-Q plot: {len(basin)} points')
            
            #py.savefig(os.path.join(out, f'{name}_QQ.png'), dpi=300)
            #py.show()
            #py.close()
        # get the name of the file without the extension
        
        # calculate shapiro-wilk test, D'Agostino and Pearson's test, and dip test
        shapiro_test = stats.shapiro(basin)
        DP_test = stats.normaltest(basin)
        dip_test = diptest.diptest(basin)
        #print(dip_test)
        #print(shapiro_test)
        if shapiro_test[1] < 0.05:
            SW_normal.append(1)
        else:
            SW_normal.append(0)
        if DP_test[1] < 0.05:
            DP_normal.append(1)
        else:
            DP_normal.append(0)
        
        # Dip test suggests minor bimodality if p-value < 0.1, significant bimodality if p-value < 0.05, therefore including both as 0 (no bimodality), 1 (Strong bimodal), or 2 (weak bimodal)
        if dip_test[1] > 0.05 and dip_test[1] < 0.1:
            bimodal.append(2)
        elif dip_test[1] < 0.05:
            bimodal.append(1)
        else:
            bimodal.append(0)
        #print(shapiro_test)
        #print(DP_test)

        # calculate the first and third quartiles
        Q1_value = np.percentile(basin, 25)
        Q3_value = np.percentile(basin, 75)
        Q1.append(Q1_value)
        Q3.append(Q3_value)

        # append the results to the lists
        B_name.append(name)
        statistic.append(shapiro_test[0])
        p_value.append(shapiro_test[1])
        DP_statistic.append(DP_test[0])
        DP_p_value.append(DP_test[1])
        dip_test_stat.append(dip_test[0])
        dip_test_p_value.append(dip_test[1])
        
        

        
        
    

/Users/samueltaylor/Documents/GitHub/PHA_burn_severity/.venv/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 19739.
  res = hypotest_fun_out(*samples, **kwds)
/Users/samueltaylor/Documents/GitHub/PHA_burn_severity/.venv/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 16335.
  res = hypotest_fun_out(*samples, **kwds)
/Users/samueltaylor/Documents/GitHub/PHA_burn_severity/.venv/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 6602.
  res = hypotest_fun_out(*samples, **kwds)
/Users/samueltaylor/Documents/GitHub/PHA_burn_severity/.venv/lib/python3.9/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, co

In [44]:

# create a DataFrame from the lists
df = pd.DataFrame({
    'Basin_ID': B_name,
    'SW_Statistic': statistic,
    'SW_P_value': p_value,
    'SW_Normal': SW_normal,
    'DP_Statistic': DP_statistic,
    'DP_P_value': DP_p_value,
    'DP_Normal': DP_normal,
    'Dip_Statistic': dip_test_stat,
    'Dip_P_value': dip_test_p_value,
    'Bimodal': bimodal,
    'Q1': Q1,
    'Q3': Q3
})

In [45]:
df

,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal,Q1,Q3
0,10009,0.927906,3.681231e-04,1,25.723897,2.594937e-06,1,0.061124,0.033573,1,0.400078,0.676352
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0,0.316569,0.609754
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0,0.416359,0.504033
3,10060,0.912353,5.436477e-04,1,3.475222,1.759402e-01,0,0.041927,0.667167,0,0.381505,0.477545
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0,0.429042,0.529377
...,...,...,...,...,...,...,...,...,...,...,...,...
1731,9942,0.911990,2.118356e-07,1,213.922293,3.526667e-47,1,0.050437,0.012093,1,0.044262,0.382404
1732,9959,0.920445,7.391726e-07,1,32.367704,9.363588e-08,1,0.044982,0.044959,1,0.658777,0.760692
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0,0.556928,0.799709
1734,9985,0.967064,1.493179e-09,1,40.970782,1.268550e-09,1,0.011085,0.968856,0,0.521736,0.709120


In [46]:
df.to_csv(os.path.join(out, 'normality_test_results.csv'), index=False)

In [17]:
SW_df = df[df['SW_Normal'] == 1]
SW_df


,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
0,10009,0.927906,3.681231e-04,1,25.723897,2.594937e-06,1,0.061124,0.033573,1
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0
3,10060,0.912353,5.436477e-04,1,3.475222,1.759402e-01,0,0.041927,0.667167,0
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0
...,...,...,...,...,...,...,...,...,...,...
1731,9942,0.911990,2.118356e-07,1,213.922293,3.526667e-47,1,0.050437,0.012093,1
1732,9959,0.920445,7.391726e-07,1,32.367704,9.363588e-08,1,0.044982,0.044959,1
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0
1734,9985,0.967064,1.493179e-09,1,40.970782,1.268550e-09,1,0.011085,0.968856,0


In [8]:
DP_df = df[df['DP_Normal'] == 1]
DP_df

,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
0,10009,0.927906,3.681231e-04,1,25.723897,2.594937e-06,1,0.061124,0.033573,1
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0
5,10107,0.954686,1.572260e-06,1,86.759039,1.447152e-19,1,0.031873,0.107492,0
...,...,...,...,...,...,...,...,...,...,...
1731,9942,0.911990,2.118356e-07,1,213.922293,3.526667e-47,1,0.050437,0.012093,1
1732,9959,0.920445,7.391726e-07,1,32.367704,9.363588e-08,1,0.044982,0.044959,1
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0
1734,9985,0.967064,1.493179e-09,1,40.970782,1.268550e-09,1,0.011085,0.968856,0


In [9]:
nobdf = df[df['Bimodal'] == 0]
strbdf = df[df['Bimodal'] == 1]
wkbdf = df[df['Bimodal'] == 2]

In [10]:
print(len(nobdf))
print(len(strbdf))
print(len(wkbdf))


1510
203
23


In [11]:
x = len(strbdf) + len(wkbdf)
print(x)

226


In [12]:
#create datframe of skewed data, pull data from SW_Normal == 1 and Bimodal == 0

SWskewdf = df[(df['SW_Normal'] == 1) & (df['Bimodal'] == 0)]
DPskewdf = df[(df['DP_Normal'] == 1) & (df['Bimodal'] == 0)]
SWskewdf

,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0
3,10060,0.912353,5.436477e-04,1,3.475222,1.759402e-01,0,0.041927,0.667167,0
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0
5,10107,0.954686,1.572260e-06,1,86.759039,1.447152e-19,1,0.031873,0.107492,0
...,...,...,...,...,...,...,...,...,...,...
1729,9903,0.930187,8.375209e-04,1,10.395261,5.529651e-03,1,0.042743,0.469034,0
1730,9906,0.984439,4.395452e-02,1,4.320410,1.153015e-01,0,0.029931,0.315710,0
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0
1734,9985,0.967064,1.493179e-09,1,40.970782,1.268550e-09,1,0.011085,0.968856,0


In [13]:
DPskewdf

,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0
5,10107,0.954686,1.572260e-06,1,86.759039,1.447152e-19,1,0.031873,0.107492,0
6,10109,0.983540,3.408481e-42,1,666.399359,1.964346e-145,1,0.001360,0.997106,0
...,...,...,...,...,...,...,...,...,...,...
1728,9897,0.960243,1.182618e-03,1,7.560179,2.282065e-02,1,0.022927,0.957650,0
1729,9903,0.930187,8.375209e-04,1,10.395261,5.529651e-03,1,0.042743,0.469034,0
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0
1734,9985,0.967064,1.493179e-09,1,40.970782,1.268550e-09,1,0.011085,0.968856,0


In [14]:
SW_tot = (len(df) - x) - len(SWskewdf)
DP_tot = (len(df) - x) - len(DPskewdf)
SW_tot 


328

In [18]:
DP_tot

419

In [19]:
ROC_m1 = r'/Volumes/PhD/imagery/masters/ROC/M1_CT_ROC.shp'
ROC_m1 = gpd.read_file(ROC_m1)
ROC_m1

,BASIN_ID,Shape_Leng,Shape_Area,Segment_ID,Acc_Cl,Perim_Buff,InsidePeri,OutsidePer,Fire_ID,Fire_Segme,...,distance_t,RI_index,RI,Mean_dNBR,Med_dNBR,PF_dNBR,M1P_Mean,M1P_Med,M1p_PF,geometry
0,17499,7120.0,1.429800e+06,17499,1.0,10,100,0,thm2017,thm2017_17499,...,2055.346175,23,5.08,0.471854,0.477169,0.459052,0.449618,0.454098,0.438860,"POLYGON ((295619.84 3816329.559, 295619.84 381..."
1,3115,1120.0,4.180000e+04,3115,1.0,10,100,10,thm2017,thm2017_3115,...,1595.604025,29,7.11,0.072363,0.014718,-0.019005,0.148129,0.116710,0.101142,"MULTIPOLYGON (((298029.84 3825079.559, 298029...."
2,3589,4020.0,4.313000e+05,3589,1.0,10,100,0,thm2017,thm2017_3589,...,1881.463927,29,7.11,0.216498,0.051373,-0.020064,0.301476,0.164258,0.122691,"POLYGON ((297059.84 3824619.559, 297069.84 382..."
3,5683,4420.0,5.659000e+05,5683,1.0,10,101,12,thm2017,thm2017_5683,...,1702.372795,21,12.95,0.692429,0.702799,0.724929,0.999180,0.999251,0.999382,"POLYGON ((283709.84 3822919.559, 283689.84 382..."
4,5920,16620.0,7.486600e+06,5920,1.0,10,100,0,thm2017,thm2017_5920,...,1525.053372,37,6.10,0.605956,0.651802,0.753237,0.848432,0.870989,0.910869,"POLYGON ((303389.84 3819559.559, 303389.84 381..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,15065,1720.0,9.550000e+04,15065,1.0,10,100,0,thm2017,thm2017_15065,...,1173.707451,10,9.14,0.164376,0.111513,0.055359,0.313190,0.248063,0.189564,"POLYGON ((289119.84 3817119.559, 289079.84 381..."
733,35852,2080.0,8.360000e+04,35852,1.0,10,101,12,thm2017,thm2017_35852,...,2151.287445,25,7.37,0.284851,0.279801,0.261897,0.473049,0.466838,0.444912,"MULTIPOLYGON (((278059.84 3804989.559, 278069...."
734,36553,1460.0,4.570000e+04,36553,1.0,10,100,11,thm2017,thm2017_36553,...,1675.063365,25,7.37,0.348485,0.339397,0.331989,0.468316,0.457159,0.448097,"MULTIPOLYGON (((277049.84 3804469.559, 277049...."
735,36955,2380.0,1.098000e+05,36955,1.0,10,100,11,thm2017,thm2017_36955,...,1352.291607,25,7.37,0.208930,0.202108,0.175856,0.281668,0.274903,0.249830,"POLYGON ((277699.84 3804059.559, 277689.84 380..."


In [21]:
ROC_basins = ROC_m1['BASIN_ID'].tolist()
#ROC_basins

In [14]:
df

,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
0,10009,0.927906,3.681231e-04,1,25.723897,2.594937e-06,1,0.061124,0.033573,1
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0
3,10060,0.912353,5.436477e-04,1,3.475222,1.759402e-01,0,0.041927,0.667167,0
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0
...,...,...,...,...,...,...,...,...,...,...
1731,9942,0.911990,2.118356e-07,1,213.922293,3.526667e-47,1,0.050437,0.012093,1
1732,9959,0.920445,7.391726e-07,1,32.367704,9.363588e-08,1,0.044982,0.044959,1
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0
1734,9985,0.967064,1.493179e-09,1,40.970782,1.268550e-09,1,0.011085,0.968856,0


In [22]:
#change Basin_ID column in df to string
df['Basin_ID'] = df['Basin_ID'].astype(int)
df

,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
0,10009,0.927906,3.681231e-04,1,25.723897,2.594937e-06,1,0.061124,0.033573,1
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0
3,10060,0.912353,5.436477e-04,1,3.475222,1.759402e-01,0,0.041927,0.667167,0
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0
...,...,...,...,...,...,...,...,...,...,...
1731,9942,0.911990,2.118356e-07,1,213.922293,3.526667e-47,1,0.050437,0.012093,1
1732,9959,0.920445,7.391726e-07,1,32.367704,9.363588e-08,1,0.044982,0.044959,1
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0
1734,9985,0.967064,1.493179e-09,1,40.970782,1.268550e-09,1,0.011085,0.968856,0


In [23]:
# Filter the DataFrame to include only basins if basin_id is equal to any of the basins in the ROC_basins list
filtered_df = df[df['Basin_ID'].isin(ROC_basins)]
filtered_df


,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0
3,10060,0.912353,5.436477e-04,1,3.475222,1.759402e-01,0,0.041927,0.667167,0
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0
7,10155,0.956637,8.430536e-20,1,144.968298,3.315386e-32,1,0.007702,0.906938,0
...,...,...,...,...,...,...,...,...,...,...
1725,9865,0.945665,7.641928e-03,1,3.039715,2.187431e-01,0,0.060413,0.077533,2
1730,9906,0.984439,4.395452e-02,1,4.320410,1.153015e-01,0,0.029931,0.315710,0
1732,9959,0.920445,7.391726e-07,1,32.367704,9.363588e-08,1,0.044982,0.044959,1
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0


In [34]:
# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(os.path.join(out, 'normality_test_results_filtered.csv'), index=False)

In [25]:
fSWskewdf = filtered_df[(filtered_df['SW_Normal'] == 1) & (filtered_df['Bimodal'] == 0)]
fDPskewdf = filtered_df[(filtered_df['DP_Normal'] == 1) & (filtered_df['Bimodal'] == 0)]
fBimodal = filtered_df[filtered_df['Bimodal'] == 2]
fSWskewdf

,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0
3,10060,0.912353,5.436477e-04,1,3.475222,1.759402e-01,0,0.041927,0.667167,0
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0
7,10155,0.956637,8.430536e-20,1,144.968298,3.315386e-32,1,0.007702,0.906938,0
...,...,...,...,...,...,...,...,...,...,...
1722,9836,0.962113,1.028833e-09,1,94.651537,2.796918e-21,1,0.019230,0.276683,0
1723,9853,0.959738,3.757972e-04,1,8.232425,1.630616e-02,1,0.017999,0.992428,0
1730,9906,0.984439,4.395452e-02,1,4.320410,1.153015e-01,0,0.029931,0.315710,0
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0


In [26]:
fDPskewdf

,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
1,10048,0.985049,1.287702e-05,1,12.362344,2.068002e-03,1,0.013168,0.773796,0
2,10058,0.891842,7.096566e-06,1,10.681829,4.791486e-03,1,0.025264,0.990390,0
4,10104,0.853367,1.239101e-08,1,27.076409,1.319570e-06,1,0.028374,0.848372,0
7,10155,0.956637,8.430536e-20,1,144.968298,3.315386e-32,1,0.007702,0.906938,0
8,10194,0.941233,4.717875e-06,1,13.631545,1.096346e-03,1,0.021895,0.913145,0
...,...,...,...,...,...,...,...,...,...,...
1715,9740,0.959827,3.730289e-11,1,58.115628,2.400776e-13,1,0.008393,0.995892,0
1722,9836,0.962113,1.028833e-09,1,94.651537,2.796918e-21,1,0.019230,0.276683,0
1723,9853,0.959738,3.757972e-04,1,8.232425,1.630616e-02,1,0.017999,0.992428,0
1733,9974,0.958141,9.943072e-25,1,147.474170,9.470892e-33,1,0.004796,0.992495,0


In [27]:
fBimodal


,Basin_ID,SW_Statistic,SW_P_value,SW_Normal,DP_Statistic,DP_P_value,DP_Normal,Dip_Statistic,Dip_P_value,Bimodal
84,11469,0.832853,3.229132e-06,1,35.775304,1.704085e-08,1,0.064483,0.087263,2
133,11986,0.911319,5.803233e-05,1,7.741431,2.084345e-02,1,0.055920,0.071717,2
184,13132,0.964766,5.968618e-10,1,88.309516,6.665494e-20,1,0.022361,0.070037,2
349,20254,0.894595,4.807991e-14,1,34.416817,3.361114e-08,1,0.027665,0.088659,2
514,23645,0.741661,3.559324e-26,1,121.896683,3.392123e-27,1,0.022970,0.093859,2
535,23972,0.962683,2.402090e-19,1,132.357582,1.815166e-29,1,0.012724,0.098585,2
741,26985,0.980878,3.970108e-03,1,7.674469,2.155312e-02,1,0.032323,0.097148,2
943,31591,0.942872,4.511664e-20,1,136.694401,2.075812e-30,1,0.015218,0.090225,2
1281,38408,0.973212,8.256846e-13,1,43.832232,3.033554e-10,1,0.015856,0.083788,2
1295,3859,0.966571,2.787874e-02,1,11.205273,3.688127e-03,1,0.051759,0.091246,2


In [28]:
fnobdf = filtered_df[filtered_df['Bimodal'] == 0]
fstrbdf = filtered_df[filtered_df['Bimodal'] == 1]
fwkbdf = filtered_df[filtered_df['Bimodal'] == 2]


In [29]:
print(len(fnobdf))
print(len(fstrbdf))
print(len(fwkbdf))

651
70
16


In [33]:
tot_sub_Sw = len(fSWskewdf) + len(fstrbdf) + len(fwkbdf)
SW_pct = (tot_sub_Sw / len(filtered_df)) * 100
print(tot_sub_Sw, SW_pct)
tot_sub_DP = len(fDPskewdf) + len(fstrbdf) + len(fwkbdf)
DP_pct = (tot_sub_DP / len(filtered_df)) * 100
print(tot_sub_DP, DP_pct)

607 82.36092265943013
567 76.93351424694708
